# Practical 5: Neural Network

### In this practical
1. [Resuming from practical 4](#resume)
2. [Building your first neural network model](#build)
3. [Understanding your neural network model](#viz)
4. [Finding optimal hyperparameters with GridSearchCV](#gridsearch)
5. [Feature selection](#fselect)
6. [Comparing models](#comparison)

---

### Important Changelog:
* (25/07/2017) Made tutorial notes public.

This practical introduces neural network mining in Python. Similar with previous practicals, our objective is to build a neural network to classify the lapsing donors based on their responses to the greeting card mailing campaign conducted by the national veterans' organisation. We will continue using **PVA97NK** dataset to predict **TARGETB**.

With its exotic sounding name, a neural network model is often regarded as mysterious yet powerful predictive tool. Perhaps surprisingly, the most typical form of neural network, in fact, is a natural extension of regression model. This form of neural network is called **multilayer perceptron**, which is the subject of our practical today.

<img src="resources/brain_vs_perceptron.png" width="300">
*Brain neuron vs a perceptron*

Whereas the strength of regression models is making decision in data with linear relationships, the strength of multi-layer perceptrons is their ability to go beyond linear relationships and model non-linear relationships in data.

<img src="resources/mlp.png" width="400">
*Multilayer perceptron structure*

Multilayer perceptron models were originally inspired by structure and interconnections between neurons in brain. They are often represented using network diagram instead of an equation. The basic model form arranges neurons in layers. The first layer, called the **input layer**, connects to one or more **hidden layers**, which in turn, connect to the final layer called **target/output layer**. Connections between each layer correspond to certain set of weights, which like regression model, are optimised during training process.

At the end of this practical, we would have built a number of predictive models. In practice, given a new dataset, data science professionals will build and experiment with many different models. Thus, it is important to understand how to compare these models and choose the best model. The second part of this practical guides you to assessing all of the models we have built so far - decision trees, logistic regressions and neural networks.

In cases such as the financial and health domains, performance of a predictive model is crucial. To achieve even better performance, multiple models can be combined in together to achieve higher predictive performance than individual models. This approach is called **ensemble modeling** and it will be covered in the last part of this practical.

## 1. Resuming from practical 4<a name="resume"></a>
Similar with practical 3 and 4, we will reuse the code for data preprocessing. Just as regression models, neural networks are sensitive to data on different scales, thus we will also perform standarization on the dataset:

In [1]:
# libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
from dm_tools import data_prep
from sklearn.preprocessing import StandardScaler

# preprocessing step
df = data_prep()

# random state
rs = 10

# train test split
y = df['TargetB']
X = df.drop(['TargetB'], axis=1)
X_mat = X.as_matrix()
X_train, X_test, y_train, y_test = train_test_split(X_mat, y, test_size=0.3, stratify=y, random_state=rs)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train, y_train)
X_test = scaler.transform(X_test)

## 2. Building your first neural network model

Start by importing your neural network from the library. In `sklearn`, neural network classifier is implemented in `MLPClassifier`, short for multilayer perceptron classifier.

In [2]:
from sklearn.neural_network import MLPClassifier

Let's train our first MLPClassifier. Initiate the model without any additional parameter (other than the random state for consistency), fit it to the training data and test its performance on test data.

In [3]:
model = MLPClassifier(random_state=rs)
model.fit(X_train, y_train)

print("Train accuracy:", model.score(X_train, y_train))
print("Test accuracy:", model.score(X_test, y_test))

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

print(model)

Train accuracy: 0.882890855457
Test accuracy: 0.546455609085
             precision    recall  f1-score   support

          0       0.55      0.52      0.54      1453
          1       0.54      0.57      0.56      1453

avg / total       0.55      0.55      0.55      2906

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=10, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)


This default neural network performed alright, with 0.546 accuracy score on the test data. Similar with the first decision tree that we trained in practical 3, you should notice that the training accuracy is much higher than the test accuracy. This is an indication the model overfits to the training data, which we will fix through GridSearch tuning and dimensionality reduction techniques.

### 2.1. Solver and activation function

As discussed in the lecture, finding the best combination of weights in neural networks is a significant search problem. The algorithm used to find this optimal weight set is called **solver** in sklearn, and the most common one is gradient descent. Gradient descent starts with a set of randomly generated weights. In each iteration of gradient descent, predictions are made on X_train and the error value (cost) is computed. The weight set is then altered to reduce this error value. Each iteration is called an epoch. To stop gradient descent iterations, either one of **maximum iterations**, **minimum error threshold** or **convergence reached** (error is not improved over a certain number of epochs) strategies is used, with combination of max iterations and convergence being the most commonly used.

In sklearn, if the neural network does not achieve convergence before maximum iteration, it will raise a "convergence is not reached" warning message. Fortunately, our first neural network reached convergence before maximum iteration limit (default to 200). If you see the message raised, the `max_iter` hyperparameter of the neural network should be increased. However, if even with very large number of maximum iteration the neural network still fails to reach convergence, this might indicate a problem with your error computation.

The following code demonstrates the warning message raised by a MLP classifier with only 100 `max_iter`. Notice that the training accuracy is lower, as the solver has not finished optimising on training data.

In [5]:
model = MLPClassifier(max_iter=100, random_state=rs)
model.fit(X_train, y_train)

print("Train accuracy:", model.score(X_train, y_train))
print("Test accuracy:", model.score(X_test, y_test))

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

print(model)

Train accuracy: 0.82197640118
Test accuracy: 0.55196145905
             precision    recall  f1-score   support

          0       0.55      0.55      0.55      1453
          1       0.55      0.55      0.55      1453

avg / total       0.55      0.55      0.55      2906

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=100, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=10, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)


C:\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


Observe the MLPClassifier object hyperparameters printed out above. You should see that the `solver` hyperparameter is set by default to **adam** (stands for adaptive moment estimation). Adam is an extension of gradient descent, designed to speed up the training process and be more computationaly efficient. Adam is the solver algorithm of choice for many deep neural networks for its efficiency and we will use adam instead of normal gradient descent here.

[A great explaination of adam](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/)

Another important hyperparameter to observe is `activation`, which refers to activation function used in hidden layers of the neural network. There are a number of options to use, including:
* **identity**: $f(x) = x$, no transformation
* **tanh**: $f(x) = {tanh}(x)$, used in lecture
* **sigmoid**: $f(x) = \frac{1}{1 + e^{h(\theta, x)}}$, used in logistic regression
* **relu - rectified linear unit**: $f(x) = max(x,0)$, default option in sklearn

Identity function will change neural network into linear model, thus it is not commonly used. In the past, tanh and sigmoid are very popular, however, recent research suggested using **relu** in neural network can produce similarly accurate result at much lower training time. Therefore, we will use relu as our activation function in this practical.

## 4. Finding optimal hyperparameter with GridSearchCV

Once we trained our first neural network, we will find the optimal hyperparameters using GridSearchCV. Neural network is harder to tune than decision trees or regression models due to relatively many type parameters and slow training process. In this practical, we will focus on tuning two parameters:
1. `hidden_layer_sizes`: It has values of tuples, and within each tuple, element i-th represent the number of neurons contained in each hidden layer.
2. `alpha`: L2 regularization parameter used in each neuron's activation function.

Start by tuning the hidden layer sizes. There is no official guideline on how many neurons we should have in each layer, but for most data mining tasks a single hidden layer with neurons no more than the number of input variables and no less than output neurons (binary classification task, hence 1) is sufficient.

> #### Deep Learning
> You might have heard of deep learning, which is process of building very complex neural networks (up to hundreds of layers and thousands of neurons, hence **deep**). Deep neural networks are typically used for complex tasks, like image recognition, Siri-like voice assistant, machine translation and self-driving tasks.

See how many input features we have by printing out the train shape.

In [6]:
print(X_train.shape)

(6780, 85)


With 85 features, we will start tuning with one hidden layer of 5 to 85 neurons, increment of 20. This might take a bit of time.

In [7]:
params = {'hidden_layer_sizes': [(x,) for x in range(5, 86, 20)]}

cv = GridSearchCV(param_grid=params, estimator=MLPClassifier(random_state=rs), cv=10, n_jobs=-1)
cv.fit(X_train, y_train)

print("Train accuracy:", cv.score(X_train, y_train))
print("Test accuracy:", cv.score(X_test, y_test))

y_pred = cv.predict(X_test)
print(classification_report(y_test, y_pred))

print(cv.best_params_)

Train accuracy: 0.631120943953
Test accuracy: 0.557123193393
             precision    recall  f1-score   support

          0       0.56      0.57      0.56      1453
          1       0.56      0.54      0.55      1453

avg / total       0.56      0.56      0.56      2906

{'hidden_layer_sizes': (5,)}


The output of this GridSearchCV returns 5 neurons as the optimal number of neurons in the hidden layer. From the past practicals and this output, it seems like less complex models (smaller trees, smaller feature set) tend to generalise on this dataset. Based on this information, we should tune around the lower number of neurons.

In [8]:
params = {'hidden_layer_sizes': [(3,), (5,), (7,), (9,)]}

cv = GridSearchCV(param_grid=params, estimator=MLPClassifier(random_state=rs), cv=10, n_jobs=-1)
cv.fit(X_train, y_train)

print("Train accuracy:", cv.score(X_train, y_train))
print("Test accuracy:", cv.score(X_test, y_test))

y_pred = cv.predict(X_test)
print(classification_report(y_test, y_pred))

print(cv.best_params_)

Train accuracy: 0.605457227139
Test accuracy: 0.557467309016
             precision    recall  f1-score   support

          0       0.55      0.59      0.57      1453
          1       0.56      0.53      0.54      1453

avg / total       0.56      0.56      0.56      2906

{'hidden_layer_sizes': (3,)}


We now have the optimal value for neuron count in our hidden layer. Next, we will tune the second hyperparameter, which is `alpha`. The default value for `alpha` is `0.0001`, thus we will try `alpha` values around this number.

In [9]:
params = {'hidden_layer_sizes': [(3,), (5,), (7,), (9,)], 'alpha': [0.01,0.001, 0.0001, 0.00001]}

cv = GridSearchCV(param_grid=params, estimator=MLPClassifier(random_state=rs), cv=10, n_jobs=-1)
cv.fit(X_train, y_train)

print("Train accuracy:", cv.score(X_train, y_train))
print("Test accuracy:", cv.score(X_test, y_test))

y_pred = cv.predict(X_test)
print(classification_report(y_test, y_pred))

print(cv.best_params_)

Train accuracy: 0.606194690265
Test accuracy: 0.558499655884
             precision    recall  f1-score   support

          0       0.56      0.59      0.57      1453
          1       0.56      0.53      0.54      1453

avg / total       0.56      0.56      0.56      2906

{'alpha': 0.01, 'hidden_layer_sizes': (3,)}


The GridSearch returned a hidden layer of 3 neurons and alpha value of 0.01 as the optimal hyperparameters.

## 5. Dimensionality reduction

In this section, we will try to improve performance of the model through dimensionality reduction and transformation techniques covered last week.

### 5.2. Log transformation

Am actually not sure of log will benefit NN.

In [15]:
import numpy as np

# list columns to be transformed
columns_to_transform = ['GiftAvg36', 'GiftAvgAll', 'GiftAvgCard36', 'GiftAvgLast',
                        'GiftCnt36', 'GiftCntAll', 'GiftCntCard36', 'GiftCntCardAll']

# copy the dataframe
df_log = df.copy()

# transform the columns with np.log
for col in columns_to_transform:
    df_log[col] = df_log[col].apply(lambda x: x+1)
    df_log[col] = df_log[col].apply(np.log)
    
# create X, y and train test data partitions
y_log = df_log['TargetB']
X_log = df_log.drop(['TargetB'], axis=1)
X_mat_log = X_log.as_matrix()
X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(X_mat_log, y_log, test_size=0.3, stratify=y_log, 
                                                                    random_state=rs)

# standardise them again
scaler_log = StandardScaler()
X_train_log = scaler_log.fit_transform(X_train_log, y_train_log)
X_test_log = scaler_log.transform(X_test_log)

In [18]:
params = {'hidden_layer_sizes': [(3,), (5,), (7,), (9,)], 'alpha': [0.01,0.001, 0.0001, 0.00001]}

cv = GridSearchCV(param_grid=params, estimator=MLPClassifier(random_state=rs), cv=10, n_jobs=-1)
cv.fit(X_train_log, y_train_log)

print("Train accuracy:", cv.score(X_train_log, y_train_log))
print("Test accuracy:", cv.score(X_test_log, y_test_log))

y_pred = cv.predict(X_test_log)
print(classification_report(y_test, y_pred))

print(cv.best_params_)

Train accuracy: 0.615634218289
Test accuracy: 0.568479008947
             precision    recall  f1-score   support

          0       0.58      0.50      0.54      1453
          1       0.56      0.64      0.60      1453

avg / total       0.57      0.57      0.57      2906

{'alpha': 0.01, 'hidden_layer_sizes': (5,)}


### 5.2. Recursive Feature Elimination

Next, we will try to reduce the feature set size using RFE. We will need a base elimination model and RFE requires type of model that assigns weight/feature importance to each feature (like regression/decision tree). Unfortunately, neural networks provide neither, thus we will try to use LogisticRegression as the base elimination model.

In [10]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression

rfe = RFECV(estimator = LogisticRegression(random_state=rs), cv=10)
rfe.fit(X_train, y_train)

print(rfe.n_features_)

40


Similar with previous practical, the RFE with logistic regression has selected 40 features as the best set of features. Next, tune an `MLPClassifier` with the transformed data set as training data.

In [11]:
X_train_rfe = rfe.transform(X_train)
X_test_rfe = rfe.transform(X_test)

# step = int((X_train_rfe.shape[1] + 5)/5);
params = {'hidden_layer_sizes': [(3,), (5,), (7,), (9,)], 'alpha': [0.01,0.001, 0.0001, 0.00001]}

cv = GridSearchCV(param_grid=params, estimator=MLPClassifier(random_state=rs), cv=10, n_jobs=-1)
cv.fit(X_train_rfe, y_train)

print("Train accuracy:", cv.score(X_train_rfe, y_train))
print("Test accuracy:", cv.score(X_test_rfe, y_test))

y_pred = cv.predict(X_test_rfe)
print(classification_report(y_test, y_pred))

print(cv.best_params_)

Train accuracy: 0.611799410029
Test accuracy: 0.563661390227
             precision    recall  f1-score   support

          0       0.56      0.56      0.56      1453
          1       0.56      0.57      0.57      1453

avg / total       0.56      0.56      0.56      2906

{'alpha': 0.0001, 'hidden_layer_sizes': (7,)}


The RFE selected feature set showed improvements over the original data set. We managed to bring train/test accuracy closer and produce a model that generalise better.

In addition to running RFE over the original preprocessed dataset, we could also use RFE to select best features from the log transformed dataset. Try it with the following code.

In [19]:
# running RFE + log transformation
rfe = RFECV(estimator = LogisticRegression(random_state=rs), cv=10)
rfe.fit(X_train_log, y_train_log) # run the RFECV on log transformed dataset

# comparing how many variables before and after
print("Original feature set", X_train_log.shape[1])
print("Number of features after elimination", rfe.n_features_)

# select features from log transformed dataset
X_train_sel_log = rfe.transform(X_train_log)
X_test_sel_log = rfe.transform(X_test_log)

# init grid search CV on transformed dataset
params = {'hidden_layer_sizes': [(3,), (5,), (7,), (9,)], 'alpha': [0.01,0.001, 0.0001, 0.00001]}
cv = GridSearchCV(param_grid=params, estimator=MLPClassifier(random_state=rs), cv=10, n_jobs=-1)
cv.fit(X_train_sel_log, y_train_log)

# test the best model
print("Train accuracy:", cv.score(X_train_sel_log, y_train_log))
print("Test accuracy:", cv.score(X_test_sel_log, y_test_log))

y_pred_log = cv.predict(X_test_sel_log)
print(classification_report(y_test_log, y_pred_log))

# print parameters of the best model
print(cv.best_params_)

Original feature set 85
Number of features after elimination 34
Train accuracy: 0.59587020649
Test accuracy: 0.56159669649
             precision    recall  f1-score   support

          0       0.56      0.60      0.58      1453
          1       0.57      0.52      0.54      1453

avg / total       0.56      0.56      0.56      2906

{'alpha': 1e-05, 'hidden_layer_sizes': (3,)}


This decision tree classifier RFE selects 8 features, which the tuned neural network produces slightly different performance than the LogisticRegression RFE. Both approaches are valid and you could try both when working with a new data set.

### 5.2. Principle Component Analysis

The second feature reduction technique that we will try is principle component analysis, or PCA. As mentioned last week, PCA is a technique that finds underlying variables (known as principal components) that best differentiate your data points. The idea of PCA is to reduce the number of features while still retaining the variance/pattern in the feature set. As last week, we will set our variance threshold at 95%.

In [21]:
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(X_train_log)

sum_var = 0
for idx, val in enumerate(pca.explained_variance_ratio_):
    sum_var += val
    if (sum_var >= 0.95):
        print("N components with > 95% variance =", idx+1)
        break

N components with > 95% variance = 65


Similar with last week, 65 components with cumulative > 95% variance are selected. Train and test on this transformed dataset.

In [22]:
pca = PCA(n_components=66)
X_train_pca = pca.fit_transform(X_train_log)
X_test_pca = pca.transform(X_test_log)

params = {'hidden_layer_sizes': [(3,), (5,), (7,), (9,)], 'alpha': [0.01,0.001, 0.0001, 0.00001]}

cv = GridSearchCV(param_grid=params, estimator=MLPClassifier(random_state=rs), cv=10, n_jobs=-1)
cv.fit(X_train_pca, y_train)

print("Train accuracy:", cv.score(X_train_pca, y_train_log))
print("Test accuracy:", cv.score(X_test_pca, y_test_log))

y_pred = cv.predict(X_test_pca)
print(classification_report(y_test_log, y_pred))

print(cv.best_params_)

Train accuracy: 0.606342182891
Test accuracy: 0.554714384033
             precision    recall  f1-score   support

          0       0.55      0.62      0.58      1453
          1       0.56      0.49      0.52      1453

avg / total       0.56      0.55      0.55      2906

{'alpha': 0.0001, 'hidden_layer_sizes': (3,)}


The result shows an improved performance over the original feature set. We also managed to reduce the feature set size to only 66, which shorten the training process. However, compared to the RFE selected feature set, PCA still produces slightly worse performance.

### 5.3. Selecting using decision tree

Lastly, we will use decision tree and feature importance produces from the model to perform feature selection. To start, we need to tune a decision tree with GridSearchCV.

In [18]:
from sklearn.tree import DecisionTreeClassifier

params = {'criterion': ['gini', 'entropy'],
          'max_depth': range(3, 8),
          'min_samples_leaf': range(20, 61, 10)}

cv = GridSearchCV(param_grid=params, estimator=DecisionTreeClassifier(random_state=rs), cv=10)
cv.fit(X_train, y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'criterion': ['gini', 'entropy'], 'max_depth': range(3, 10), 'min_samples_leaf': range(20, 200, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [19]:
from dm_tools import analyse_feature_importance

analyse_feature_importance(cv.best_estimator_, X.columns)

GiftAvgLast : 0.312603065255
DemMedHomeValue : 0.157362668946
GiftTimeLast : 0.102247296701
GiftAvgCard36 : 0.0894450622497
GiftCntCardAll : 0.0564975901627
PromCnt12 : 0.0509281789485
DemPctVeterans : 0.04408455517
PromCntAll : 0.0301916892275
GiftCntAll : 0.026795816426
GiftCnt36 : 0.0264996721409
GiftAvg36 : 0.0250858556392
DemMedIncome : 0.0195942277141
GiftAvgAll : 0.018633476267
PromCntCard36 : 0.0167406544809
PromCntCardAll : 0.0122122046814
PromCntCard12 : 0.0110779859909
GiftTimeFirst : 0.0
DemCluster_10 : 0.0
DemCluster_11 : 0.0
DemCluster_12 : 0.0


In [20]:
from sklearn.feature_selection import SelectFromModel

selectmodel = SelectFromModel(cv.best_estimator_, prefit=True)
X_train_sel_model = selectmodel.transform(X_train)
X_test_sel_model = selectmodel.transform(X_test)

print(X_train_sel_model.shape)

(6489, 15)


Our `analyse_feature_importance` function shows around 7 important features according to this decision tree model. With this result, `SelectFromModel` transforms the original dataset into only 7 columns. Proceed to tune a MLPClassifier with this dataset.

In [21]:
params = {'hidden_layer_sizes': [(3,), (5,), (7,), (9,)], 'alpha': [0.01,0.001, 0.0001, 0.00001]}

cv = GridSearchCV(param_grid=params, estimator=MLPClassifier(max_iter=200, random_state=rs), cv=10, n_jobs=-1)
cv.fit(X_train_sel_model, y_train)

print("Train accuracy:", cv.score(X_train_sel_model, y_train))
print("Test accuracy:", cv.score(X_test_sel_model, y_test))

y_pred = cv.predict(X_test_sel_model)
print(classification_report(y_test, y_pred))

print(cv.best_params_)

Train accuracy: 0.586839266451
Test accuracy: 0.568032530497
             precision    recall  f1-score   support

          0       0.56      0.63      0.59      1599
          1       0.58      0.50      0.54      1598

avg / total       0.57      0.57      0.57      3197

{'alpha': 0.01, 'hidden_layer_sizes': (7,)}


The test accuracy result shows improvement over the original feature set and RFE selected feature set as well. This method yields the smallest feature set yet (only 7 rather than 85 features), and again the performance is the best. We will keep this model as the best model.

## 6. Comparing Models

After 5 weeks, we have learned how to perform data preprocessing, model tuning and various dimensionality reduction techniques. While we have been pretty straight forward with our decisions and process, in real life projects, you need to test many different techniques/approaches before getting to the solution. You need to constantly expand your tool box and knowledge to be a great data miner/data scientist/machine learning engineer.

Now, let's imagine that you have built multiple models, where each uses different data preprocessing/dimensionality reduction techniques. How do you choose the best performing model? One way to do this is by comparing statistics produced by each model. I will show you how.

Firstly, let's train and tune three models of `DecisionTreeClassifier`, `LogisticRegression` and `MLPClassifier` with GridSearchCV. We will use the original feature set with no dimensionality reduction for this demonstration, but the process should not be too much different.

In [ ]:
# grid search CV for decision tree
params_dt = {'criterion': ['gini'],
          'max_depth': range(2, 5),
          'min_samples_leaf': range(40, 61, 5)}

cv = GridSearchCV(param_grid=params_dt, estimator=DecisionTreeClassifier(random_state=rs), cv=10)
cv.fit(X_train, y_train)

dt_model = cv.best_estimator_
print(dt_model)

# grid search CV for logistic regression
params_log_reg = {'C': [pow(10, x) for x in range(-6, 4)]}

cv = GridSearchCV(param_grid=params_log_reg, estimator=LogisticRegression(random_state=rs), cv=10, n_jobs=-1)
cv.fit(X_train, y_train)

log_reg_model = cv.best_estimator_
print(log_reg_model)

# grid search CV for NN
params_nn = {'hidden_layer_sizes': [(3,), (5,), (7,), (9,)], 'alpha': [0.01,0.001, 0.0001, 0.00001]}

cv = GridSearchCV(param_grid=params_nn, estimator=MLPClassifier(max_iter=500, random_state=rs), cv=10, n_jobs=-1)
cv.fit(X_train, y_train)

nn_model = cv.best_estimator_
print(nn_model)

### 6.1. Test Accuracy

Once you have them trained, there are a number of statistics that we could use for comparing models. First, accuracy of the models on test data, just like what we have used so far.

**Note**: Accuracy is a great statistics when the ratio of target classes are relatively equal, like what we have in this dataset (50% donors vs 50% non-donors). In cases where the targets are not equal (e.g. in cancer detection task where most people in the dataset will not have cancer), metrics like precision/recall/F1 from `classification_report` or Cohen's kappa are preferred.

In [ ]:
y_pred_dt = dt_model.predict(X_test)
y_pred_log_reg = log_reg_model.predict(X_test)
y_pred_nn = nn_model.predict(X_test)

print("Accuracy score on test for DT:", accuracy_score(y_test, y_pred_dt))
print("Accuracy score on test for logistic regression:", accuracy_score(y_test, y_pred_log_reg))
print("Accuracy score on test for NN:", accuracy_score(y_test, y_pred_nn))

On test accuracy score, logistic regression performs the best, followed by decision tree and neural network.

### 6.2. ROC AUC
Another metric commonly used to compare models is receiver operating characteristic (ROC) and area under curve (AUC). ROC refers to the ability of binary classifier (like what we have here) to classify with varied discrimination threshold.

Most predictive classification models produce probability of target values on a set of inputs. `LogisticRegression` and `MLPClassifier` produces real value probabilities, while `DecisionTree` has the ratio of majority classes in each leaf node. Most of the time, discrimination threshold is cap at 0.5, which means any probability prediction above 0.5 is considered as positive (and the rest negative). For more clarity, see this code below:

In [ ]:
# typical prediction
y_pred = dt_model.predict(X_test)

# probability prediction from decision tree
y_pred_proba_dt = dt_model.predict_proba(X_test)

print("Probability produced by decision tree for each class vs actual prediction on TargetB (0 = non-donor, 1 = donor). You should be able to see the default threshold of 0.5.")
print("(Probs on zero)\t(probs on one)\t(prediction made)")
# print top 10
for i in range(10):
    print(y_pred_proba_dt[i][0], '\t', y_pred_proba_dt[i][1], '\t', y_pred[i])

With this concept in mind, ROC AUC score aims to find the best model under varied threshold. To compute our ROC AUC score, use the code below.

In [ ]:
from sklearn.metrics import roc_auc_score

y_pred_proba_dt = dt_model.predict_proba(X_test)
y_pred_proba_log_reg = log_reg_model.predict_proba(X_test)
y_pred_proba_nn = nn_model.predict_proba(X_test)

roc_index_dt = roc_auc_score(y_test, y_pred_proba_dt[:, 1])
roc_index_log_reg = roc_auc_score(y_test, y_pred_proba_log_reg[:, 1])
roc_index_nn = roc_auc_score(y_test, y_pred_proba_nn[:, 1])

print("ROC index on test for DT:", roc_index_dt)
print("ROC index on test for logistic regression:", roc_index_log_reg)
print("ROC index on test for NN:", roc_index_nn)

`LogisticRegression` produces the best ROC score. This means on varied discrimination threshold, this LogReg model performs better compared to the other two models.

ROC score only tells a side of the story, however. Typically, instead of ROC score, we plot a curve to show the performance of the model on different threshold values. The curve should look something like this, and the closer the curve is to top left corner, the better the model is.

![ROC Curve](http://gim.unmc.edu/dxtests/roccomp.jpg)

Let's plot ROC curve for our models. Firstly, we need to find the false positive rate, true positive rate and thresholds used for each models. We can get it from the `roc_curve` function.

In [ ]:
from sklearn.metrics import roc_curve

fpr_dt, tpr_dt, thresholds_dt = roc_curve(y_test, y_pred_proba_dt[:,1])
fpr_log_reg, tpr_log_reg, thresholds_log_reg = roc_curve(y_test, y_pred_proba_log_reg[:,1])
fpr_nn, tpr_nn, thresholds_nn = roc_curve(y_test, y_pred_proba_nn[:,1])

Once we have these scores, plot them using `matplotlib`'s pyplot.

In [ ]:
import matplotlib.pyplot as plt

plt.plot(fpr_dt, tpr_dt, label='ROC Curve for DT {:.3f}'.format(roc_index_dt), color='red', lw=0.5)
plt.plot(fpr_log_reg, tpr_log_reg, label='ROC Curve for Log reg {:.3f}'.format(roc_index_log_reg), color='green', lw=0.5)
plt.plot(fpr_nn, tpr_nn, label='ROC Curve for NN {:.3f}'.format(roc_index_nn), color='darkorange', lw=0.5)

# plt.plot(fpr[2], tpr[2], color='darkorange',
#          lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], color='navy', lw=0.5, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

Here, you can see the curve for different models. `LogisticRegression` again has the largest curve area compared to the other two models. Thus, all three statistics that we used collectively agreed on `LogisticRegression` being the best performing model overall.

While statistics are vital, in a real project, performance is not always the priority. Some of the other aspects used to consider a best model are:
1. Interpretability: how well can humans use the model to make decisions. Decision trees (and regressions to some extent) excel at this, while neural network not so much.
2. Speed: how well can the model train and predict on large amount of data. Again decision trees and regressions are relatively fast, while neural networks take a while to train.
3. Adaptability: in some cases, you want your model to slowly adapt to the data trend. Neural networks are great for this as they can be trained using "online training", while decision trees are not so great.

## End Notes and Next Week

This week, we learned how to build, tune and explore the structure of neural network models. We also explored dimensionality reduction techniques to reduce the size of the feature set and improve performance of our neural network model. In addition, we tried numerous statistics to compare end-to-end performance of all models we have built so far.

Next week, we will have a drop-in help session where you focus on the assignment and ask your tutor questions regarding it.